In [ ]:
1+1

In [ ]:
import os
import sys
import pandas as pd
import torch
import sys
from tqdm import tqdm

In [ ]:
import rootutils

rootutils.setup_root('.', indicator=".project-root", pythonpath=True)
from experiment_analysis.utils import flatten_dataframe, get_wandb_table

In [ ]:
BASE_WEIGHT_PATH = os.environ['BASE_WEIGHT_PATH']

In [ ]:
os.makedirs(BASE_WEIGHT_PATH, exist_ok=True)

In [ ]:
USER_NAME = 'zaabik'

In [ ]:
sst2_table = flatten_dataframe(get_wandb_table(USER_NAME, 'uncertainty-estimation-sr-ft-sst2'))
sst5_table = flatten_dataframe(get_wandb_table(USER_NAME, 'uncertainty-estimation-sr-ft-sst5'))
cola_table = flatten_dataframe(get_wandb_table(USER_NAME, 'uncertainty-estimation-sr-ft-cola'))
toxigen_table = flatten_dataframe(get_wandb_table(USER_NAME, 'uncertainty-estimation-sr-ft-toxigen'))
newsgroups_table = flatten_dataframe(get_wandb_table(USER_NAME, 'uncertainty-estimation-sr-ft-newsgroups'))

In [ ]:
ckpt_callback_name = "ModelCheckpoint{'monitor': 'val/acc', 'mode': 'max', 'every_n_train_steps': 0, 'every_n_epochs': 1, 'train_time_interval': None}"

In [ ]:
def get_final_path(path):
    path = path + '/last.ckpt'
    ckpt = torch.load(path)
    path = ckpt['callbacks'][ckpt_callback_name]['best_model_path']
    epoch = path.split('epoch_')[1].split('.')[0]
    new_path = '/'.join(path.split('/')[:-1])
    return new_path + '/' + f'adapter_epoch_{epoch}'

In [ ]:
total_table = pd.concat([sst2_table, sst5_table, cola_table, toxigen_table,newsgroups_table]).reset_index(drop=True)
total_table = total_table[~total_table['test/acc'].isnull()]
total_table['dataset_name'] = total_table['data/dataset/_target_'].apply(lambda x: x.split('.')[-1]).values
total_table['best_ckpt_path'] = total_table['callbacks/model_checkpoint/dirpath'].apply(get_final_path)
total_table = total_table.sort_values(['dataset_name', 'base_architecture', 'seed'])#[['dataset_name', 'base_architecture']]
total_table['seed_number'] = total_table.groupby(['dataset_name', 'base_architecture']).cumcount()
total_table = total_table.reset_index(drop=True)

In [ ]:
all_runs = total_table.drop_duplicates(['base_architecture', 'dataset_name', 'seed']).groupby(['base_architecture', 'dataset_name'])[['name']].count()

In [ ]:
total_table.drop_duplicates(['base_architecture', 'dataset_name', 'seed']).groupby(['base_architecture', 'dataset_name'])[['name']].count()

In [ ]:
t = total_table.drop_duplicates(['base_architecture', 'dataset_name', 'seed'])

In [ ]:
assert len(all_runs[
    all_runs['name'] != 5
]) == 0

In [ ]:
model_name_mapping = {
    'Qwen/Qwen2.5-7B' : 'qwen_runs_seeds',
    'google/electra-base-discriminator' : 'electra_runs_seeds',
    'meta-llama/Llama-2-7b-hf' : 'llama_runs_seeds',
    'roberta-base' : 'roberta_runs_seeds',
}

In [ ]:
import os

In [ ]:
copy_to = BASE_WEIGHT_PATH + total_table['base_architecture'].map(model_name_mapping) +'/'+ total_table['dataset_name'] + '/' + total_table['seed_number'].astype(str)
copy_from = total_table['best_ckpt_path'] + '/*'
full_command = 'cp ' + copy_from + ' '+ copy_to

In [ ]:
for path in copy_to:
    try:
        os.makedirs(path)
    except:
        print(path)

In [ ]:
for command in tqdm(full_command):
    os.system(command)